In [2]:
import pandas as pd
import os
import requests
import json
from IPython.display import JSON
import gmplot
import pickle

In [59]:
# Yelp API pull
url = 'https://api.yelp.com/v3/businesses/search'
YELP_APIKEY=os.environ['YELP_APIKEY']

class BearerAuth(requests.auth.AuthBase):
    def __init__(self, token):
        self.token = token
    def __call__(self, r):
        r.headers["authorization"] = "Bearer " + self.token
        return r 

In [85]:
def get_business(loc,terms):
    params={
        'location':loc,
        'radius':10000,
        'price':1,
        'term':terms,
        'categories':'Food',
        'limit':50,
        'sort_by':'rating'
        }
    response=requests.get(url,params=params,auth=BearerAuth(YELP_APIKEY))
    return response.json()

In [86]:
bby_snack_yelp = get_business('49.225706, -123.001991','snack') #metrotown 
rmd_snack_yelp = get_business('49.156481, -123.143042','snack') #blundell 
van_dt_snack_yelp = get_business('49.285766, -123.130353','snack') #west-end 
van_west_snack_yelp = get_business('49.241093, -123.140106','snack') #Shaughnessy

In [3]:
pickle.dump(bby_snack_yelp,open('bby_snack_yelp.txt','wb'))
pickle.dump(rmd_snack_yelp,open('rmd_snack_yelp.txt','wb'))
pickle.dump(van_dt_snack_yelp,open('van_dt_snack_yelp.txt','wb'))
pickle.dump(van_west_snack_yelp,open('van_west_snack_yelp.txt','wb'))

NameError: name 'bby_snack_yelp' is not defined

In [7]:
bby_snack_yelp=pickle.load(open('bby_snack_yelp.txt','rb'))
rmd_snack_yelp=pickle.load(open('rmd_snack_yelp.txt','rb'))
van_dt_snack_yelp=pickle.load(open('van_dt_snack_yelp.txt','rb'))
van_west_snack_yelp=pickle.load(open('van_west_snack_yelp.txt','rb'))

In [6]:
van_west_snack_yelp

{'businesses': [{'id': '3ByjrLF8nyArUxvO6vqJCw',
   'alias': 'incognito-coffee-vancouver',
   'name': 'Incognito Coffee',
   'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/Sk58uxr56M3FkKI8KLWrpg/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/incognito-coffee-vancouver?adjust_creative=-_rtnR0u1hOegsQjj73OBg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=-_rtnR0u1hOegsQjj73OBg',
   'review_count': 95,
   'categories': [{'alias': 'coffee', 'title': 'Coffee & Tea'},
    {'alias': 'juicebars', 'title': 'Juice Bars & Smoothies'},
    {'alias': 'sandwiches', 'title': 'Sandwiches'}],
   'rating': 5.0,
   'coordinates': {'latitude': 49.2802573019756,
    'longitude': -123.120230028369},
   'transactions': [],
   'price': '$',
   'location': {'address1': '843 Seymour Street',
    'address2': '',
    'address3': '',
    'city': 'Vancouver',
    'zip_code': 'V6B 3L4',
    'country': 'CA',
    'state': 'BC',
    'display_address': ['843 Seymour Stre

In [10]:
responses=[bby_snack_yelp,rmd_snack_yelp,van_dt_snack_yelp,van_west_snack_yelp]
area=['bby','rmd','van_dt','van_west']

In [11]:
business={
    'name':[],
    'loc':[],
    'rating':[],
    'price':[],
    'area':[],
    'categories':[]
}

for ind in range(len(responses)):
    a=area[ind]
    response=responses[ind]
    for i in range(len(response['businesses'])):
        business['name'].append(response['businesses'][i]['name'])
        business['loc'].append((
            response['businesses'][i]['coordinates']['latitude'],
            response['businesses'][i]['coordinates']['longitude']))
        business['rating'].append(response['businesses'][i]['rating'])
        business['area'].append(a)
        try:
            business['price'].append(response['businesses'][i]['price'])
        except:
            business['price'].append('na')
        try:
            types=response['businesses'][i]['categories']
            res=[]
            for i in range(len(types)):
                res.append(types[i]['title'])
            business['categories'].append(res)
        except:
            business['categories'].append('na')
        
    

In [13]:
df=pd.DataFrame(data=business,columns=business.keys())
df.sort_values(by='name').head(10)

,name,loc,rating,price,area,categories
88,8 Juice,"(49.1835596, -123.133584)",3.5,$,rmd,[Juice Bars & Smoothies]
48,99 BBQ House,"(49.22916, -123.00448)",3.5,$,bby,[Barbeque]
129,A Bread Affair,"(49.2724508745628, -123.135854241387)",4.0,$,van_dt,"[Bakeries, Sandwiches]"
178,A Bread Affair,"(49.2724508745628, -123.135854241387)",4.0,$,van_west,"[Bakeries, Sandwiches]"
84,AAA Restaurant,"(49.1785598122947, -123.135279300271)",3.5,$,rmd,"[Noodles, Soup, Chinese]"
41,Al-Salam Halal Meat,"(49.2185936, -122.9597244)",4.0,$,bby,"[Butcher, Halal, Meat Shops]"
198,Angus T,"(49.2762326068102, -123.119233623147)",4.0,$,van_west,"[Cafes, Bakeries]"
76,Asia Market,"(49.207102, -123.0998727)",4.5,$,rmd,[International Grocery]
146,Asia Market,"(49.207102, -123.0998727)",4.5,$,van_dt,[International Grocery]
27,Assi Supermarket,"(49.2236199, -122.98278)",4.0,$,bby,[International Grocery]


In [26]:
pd.DataFrame(df['rating'].value_counts()).reset_index().sort_values(by='index')

,index,rating
4,3.0,7
2,3.5,20
0,4.0,102
1,4.5,63
3,5.0,8


In [161]:
def filter_loc_by_rat(rat):
    filtered_df=df[df['rating']==rat]
    loc=zip(*list(filtered_df['loc']))
    return loc

In [183]:
google_apikey=os.environ['GOOGLE_API_KEY']
gmap = gmplot.GoogleMapPlotter(49.2827,-123.1207,11,apikey=google_apikey)

gmap.scatter(*filter_loc_by_rat(5),color='red', marker=True)
gmap.scatter(*filter_loc_by_rat(4.5),color='orange',marker=True)
gmap.scatter(*filter_loc_by_rat(4),color='orange',marker=True)
gmap.scatter(*filter_loc_by_rat(3),color='yellow',marker=True)
gmap.scatter(*filter_loc_by_rat(3.5),color='yellow',marker=True)
gmap.draw('map.html')

In [184]:
google_apikey=os.environ['GOOGLE_API_KEY']
gmap = gmplot.GoogleMapPlotter(49.2827,-123.1207,10,apikey=google_apikey)

gmap.heatmap(*zip(*list(df['loc'])))
gmap.draw('heatmap.html')

In [236]:
df['categories'][0]

['Bakeries', 'Ukrainian']

In [237]:
for word in df['categories'][0]:
    if word in flavor:
        flavors.append(word)

In [238]:
flavors

['Chinese', 'Ukrainian']

In [239]:
col=[]
for i in range(df['categories'].shape[0]):
    flavors=[]
    for word in df['categories'][i]:
        if word in flavor:
            flavors.append(word)
        col.append(flavors)

In [240]:
col

[['Ukrainian'],
 ['Ukrainian'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['Chinese'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['Vietnamese'],
 ['Vietnamese'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['Latin American'],
 ['Latin American'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['Japanese'],
 ['Japanese'],
 ['Japanese'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['Chinese'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['Chinese'],
 ['Chinese'],
 ['Chinese'],
 ['Turkish'],
 ['Turkish'],
 ['Chinese'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['Malaysian', 'Singaporean', 'Chinese'],
 ['Malaysian', 'Singaporean', 'Chinese'],
 ['Malaysian', 'Singaporean', 'Chinese'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['Japanese', 'Hawaiian'],
 ['Japanese', 'Hawaiian'],
 ['Japanese', 'Hawaiian'],
 ['Chinese'],
 ['Chinese'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],


In [234]:
df['categories'].shape[0]

200

In [211]:
words=[]
for i in range(df['categories'].shape[0]):
    for w in df['categories'][i]:
        words.append(w)

In [225]:
flavor=['Asian Fusion','Chinese','French','Hawaiian','Japanese','Indonesian','Korean','Latin American',
 'Malaysian','Mexican',
 'Middle Eastern',
 'Modern European','Russian','Singaporean',
 'Taiwanese',
 'Turkish',
 'Ukrainian',
 'Vietnamese',]

In [213]:
df['flavor']

{'Airport Lounges',
 'Asian Fusion',
 'Australian',
 'Bagels',
 'Bakeries',
 'Barbeque',
 'Belgian',
 'Bookstores',
 'Breakfast & Brunch',
 'Breweries',
 'Bubble Tea',
 'Burgers',
 'Butcher',
 'Cafes',
 'Caribbean',
 'Caterers',
 'Chicken Wings',
 'Chinese',
 'Chocolatiers & Shops',
 'Coffee & Tea',
 'Coffee Roasteries',
 'Convenience Stores',
 'Cosmetics & Beauty Supply',
 'Creperies',
 'Delis',
 'Desserts',
 'Diners',
 'Discount Store',
 'Donairs',
 'Donuts',
 'Dumplings',
 'Fast Food',
 'Food Court',
 'Food Stands',
 'Food Trucks',
 'French',
 'Fruits & Veggies',
 'Gluten-Free',
 'Grocery',
 'Halal',
 'Hawaiian',
 'Head Shops',
 'Health Markets',
 'Hot Dogs',
 'Ice Cream & Frozen Yogurt',
 'Imported Food',
 'Indonesian',
 'International Grocery',
 'Japanese',
 'Juice Bars & Smoothies',
 'Kitchen Supplies',
 'Korean',
 'Latin American',
 'Macarons',
 'Malaysian',
 'Meat Shops',
 'Mediterranean',
 'Mexican',
 'Middle Eastern',
 'Modern European',
 'Noodles',
 'Nutritionists',
 'Passpo